# Blending and pasting images - Part 2 - Masks

When we blended images of the same size in Section 23 we can see that the background color of smaller image gets blended as well and affects the matching pixels of the other image. What we want to do now is to blend only the main/prominent object from one (smaller) image, without its background, into another (bigger) image. To achieve this, we need to perform several steps:
* extract ROI (region of interest) from larger image; it matches the dimensions of the smaller image
* create a grayscale (2D) mask which matches the object on the smaller image
* [this seems not necessary] convert grayscale mask into colour (3D) mask which matches (all 3) dimensions of the smaller image
* apply the 3D mask on the smaller image in order to extract only the object (foreground)
* blend ROI and foreground to get final version of ROI
* replace ROI with final ROI

The process above will be shown on the example of adding a watermark on an image.

## Blending together images of different sizes

In [ ]:
import cv2
import matplotlib.pyplot as plt

img1 = cv2.imread('../data/dog_backpack.png') # OpenCV reads images as BGR
print(type(img1))
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.imread('../data/watermark_no_copy.png')
print(type(img2))
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img2 = cv2.resize(img2, (600, 600)) # this is a small image


Let's now define a ROI (Region Of Interest) on the larger image. This is an area of larger image where we want to blend second (smaller) image into. Let's show the larger image:

In [ ]:
plt.imshow(img1)

In [ ]:
plt.imshow(img2)

Let's say this time we want to blend smaller image (`img2`) in the bottom right corner of the large image (`img1`):

In [ ]:
print(f'img1.shape = {img1.shape}')
print(f'img2.shape = {img2.shape}')
rows, columns, channels = img2.shape

### Extracting ROI

In [ ]:
# offset coordinates are top left corner of the ROI
x_offset = img1.shape[1] - img2.shape[1]
y_offset = img1.shape[0] - img2.shape[0]
roi = img1[y_offset:img1.shape[0], x_offset:img1.shape[1]]
plt.imshow(roi)

### Creating a grayscale (2D) mask

This mask has to have the same `width x height` as the (smaller) image and we want to have pixels of the main object to be white (`np.uint8` value `255` in grayscale) and the rest (background) to be black (`0`).

In [ ]:
# let's turn smaller image into greyscale
img2_grey = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
print(f'img2_grey = {img2_grey}')

# note that this color conversion actually removes 1 dimension (color channels) from the original image
print(f'img2.shape = {img2.shape}')
print(f'img2_grey.shape = {img2_grey.shape}')
plt.imshow(img2_grey) # by default imshow() uses Viridis color mapping

In [ ]:
# let's show the image in greyscale color mapping (which makes more sense now our image is grayscale)
plt.imshow(img2grey, cmap='gray')

To create a mask, we want to inverse this image actually as we want white to follow the shape of the object on the image that we want to be shown and black to be the background.

Read [Arithmetic Operations on Images](https://docs.opencv.org/master/d0/d86/tutorial_py_image_arithmetics.html).

In [ ]:
mask = cv2.bitwise_not(img2grey)
plt.imshow(mask, cmap='gray')

`mask` is a grayscale image so has only 2 dimensions:

In [ ]:
mask.shape

### Creating a colour (3D) mask [This seems unnecessary]

This mask has to have the same `width x height x depth` as the (smaller) image and we want to have pixels of the main object to be white (`np.uint8` value `255` in grayscale) and the rest (background) to be black (`0`) in each (RGB) channel.

To blend `img2` image into `img1` we need to apply this mask to each RGB channel of `img2`. 

In [ ]:
import numpy as np
white_background = np.ones(img2.shape) * 255
print(f'white_background.shape = {white_background.shape}')

Alternatively, to fill an arbitrary (N-dimensional) array with an arbitrary values we can use [numpy.full(shape, fill_value, dtype)](https://numpy.org/doc/stable/reference/generated/numpy.full.html) function which returns a new array of given `shape` and `type`, filled with `fill_value`:

In [ ]:
print(f'type(mask[0, 0]) = {type(mask[0, 0])}')
# data type has to match data type of mask elements
white_background = np.full(img2.shape, 255, dtype=np.uint8)
print(f'white_background.shape = {white_background.shape}')

We now want to combine 600x600 mask and 600x600 white background into 600x600x3 matrix. 
We can use [dst = cv2.bitwise_or(rc1, src2\[, dst\[, mask\]\])](https://docs.opencv.org/master/d2/de8/group__core__array.html#gab85523db362a4e26ff0c703793a719b4) which has the following arguments:
* `src1` - first input array or a scalar.
* `src2` - second input array or a scalar.
* `dst` - output array that has the same size and type as the input arrays (this is optional )
* `mask` - optional operation mask, 8-bit single channel array, that specifies elements of the output array to be changed.
In case of multi-channel arrays, each channel is processed independently. 

This last sentence is very important as this means that if we have 3-D array, bitwise operation is applied layer by layer (on 2-D matrices).

In [ ]:
# background
mask3d = cv2.bitwise_or(white_background, white_background, mask=mask)
print(f'mask3d.shape = {mask3d.shape}')

In [ ]:
plt.imshow(mask3d)

### Extracting the colour (3D) foreground

Here we want to get the main object displayed as it is, in the original colours and everything we don't want to show (background) to be in black. We can get this if we apply mask to all 3 channels:

In [ ]:
foreground = cv2.bitwise_or(img2, img2, mask=mask)
print(f'foreground.shape = {foreground.shape}')
plt.imshow(foreground)

### Get the final version of ROI

In [ ]:
final_roi = cv2.bitwise_or(roi, foreground)
plt.imshow(final_roi)

### Replace the original version of ROI with the final one

In [ ]:
large_img = img1
small_img = final_roi

large_img[y_offset : img1.shape[0], x_offset : img1.shape[1]] = small_img
plt.imshow(large_img)